In [21]:
import openai
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [13]:
open_ai_key = #your open_ai_key goes here
openai.api_key = open_ai_key

In [9]:
import pinecone
# initialize connection to pinecone
pinecone_key = #your pinecone_key goes here
pinecone.init(
    api_key=pinecone_key,  # app.pinecone.io (console)
    environment="eu-west1-gcp"  # next to API key in console
)
index_name = 'personalized-bot'
index = pinecone.Index(index_name)

C:\Users\arish\AppData\Roaming\pypoetry\venv\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [40]:
embed_model = "text-embedding-ada-002"
query = "Food preferences"
res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)
xq = res['data'][0]['embedding']

In [10]:
result = index.query(xq, top_k=10, include_metadata=True,namespace='text_field_9')

contexts = [item['metadata']['text'] for item in result['matches']]

In [32]:
string1 = """
Using the provided conversation data between two persons, your task is to emulate the speaking style and language of one of the individuals while answering the user's question.
Draw upon your own knowledge and training to provide relevant and informative responses, but ensure that your answers reflect the mannerisms, tone, and language used by the chosen person. 
Always respond in the language of the conversation data shared, regardless of the question asked.
Chosen person is
"""

string2 = """
{chat_history}
human:{human_input}
Chatbot:""
"""

In [33]:
current_person = "Aarish"
primer = string1 + current_person + string2

In [34]:
prompt = PromptTemplate(
    input_variables=["chat_history","human_input"],
    template=primer
)

In [35]:
llm = OpenAI(openai_api_key=open_ai_key,temperature=0)

In [36]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [37]:
llm_chain = LLMChain(
    llm=llm, 
    prompt=prompt, 
    verbose=True, 
    memory=memory,
)

In [38]:
input_query = 'kya khana hai?'
augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+ input_query

In [39]:
llm_chain.predict(human_input=augmented_query)



> Entering new LLMChain chain...
Prompt after formatting:

Using the provided conversation data between two persons, your task is to emulate the speaking style and language of one of the individuals while answering the user's question.
Draw upon your own knowledge and training to provide relevant and informative responses, but ensure that your answers reflect the mannerisms, tone, and language used by the chosen person. 
Always respond in the language of the conversation data shared, regardless of the question asked.
Chosen person is
Aarish

human:

-----

kya khana hai?
Chatbot:""


> Finished chain.


'\nAarish bhaiya humein kuch chatpata khana chahiye. Humare paas bahut saare options hai jaise Pani Puri, Bhel Puri, Dahi Puri, Aloo Tikki, Samosa aur bahut kuch. Kya aapko kuch pasand hai?'

In [45]:
import sys
print(sys.executable)

C:\Users\arish\AppData\Roaming\pypoetry\venv\Scripts\python.exe
